In [ ]:
import gaps_online as go
import gaps_online.db as db
import re
from glob import glob
import numpy as np
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path 

In [ ]:
#load calibrations
calib = go.tof.calibrations.load_calibrations(Path('/data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/20240305/'))

if len(calib.keys()) != 40: print('Oh no! Some calibrations are missing :(')
else: print('Yay! All calibration files were loaded :)')

In [ ]:
#testing if calib is loaded
print(sorted(calib.keys()))
print(len(calib.keys()))

In [ ]:
#load paths and populate empty data structures for later use

max_pulse_dict = defaultdict(list)
max_pulses = []

tof_events = []
fixed_path = "/data1/nextcloud/cra_data/data/2023_nevis/tof/138/Run138"


files = glob(f'{fixed_path}_*.tof.gaps')
files = files [-5:]
print(len(files)) #checking to make sure files was populated

In [ ]:
#pre-optimization

for fname in tqdm(files):
    data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
    evts = [go.cxx_api.TofEvent.from_tofpacket(k) for k in data]

    for ev in evts:
        rbids = ev.get_rbids()
        rbevents = [ev.get_rbevent(k) for k in rbids]

        for event_id in rbevents:
            if event_id.header.rb_id == 0: continue 
        
            key = event_id.header.rb_id
            if len(str(key)) > 2: continue #error where some ids are in the hundreds
            try:
                calibration = calib[key]
            except Exception as e:
                print(e)
                continue
            
            voltages = calibration.voltages(event_id)
    
            for channel in event_id.header.get_channels():
                if channel == 8: continue #exclude channel 9
                
                wv = np.array(voltages[channel]) #calibrating the voltages
                max_pulse = np.max(wv) 
                info = {'rbid' : key, 'channel' : channel, 'max_V' : max_pulse}
    
                max_pulses.append(info)

In [ ]:
#trying to optimize for memory

from tqdm import tqdm

for fname in tqdm(files):
    data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
    
    for ev in (go.cxx_api.TofEvent.from_tofpacket(k) for k in data):
        rbids = ev.get_rbids()
        
        for event_id in (ev.get_rbevent(k) for k in rbids):
            if event_id.header.rb_id == 0:
                continue
            
            key = event_id.header.rb_id
            if len(str(key)) > 2:
                continue
            
            try:
                calibration = calib[key]
            except KeyError:
                continue
            
            voltages = calibration.voltages(event_id)
    
            for channel in event_id.header.get_channels():
                if channel == 8:
                    continue
                
                wv = np.array(voltages[channel]) #calibrating the voltages
                max_pulse = np.max(wv) 
                info = {'rbid' : key, 'channel' : channel, 'max_V' : max_pulse}
    
                max_pulses.append(info)

In [ ]:
#trying harder to optimize for memory
files = files[-200]
# Function to chunk the list into smaller sublists
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Split files into chunks
chunk_size = len(files) // 4
file_chunks = chunk_list(files, chunk_size)

# Process each chunk
for chunk in tqdm(file_chunks):
    max_pulses_chunk = []  # Initialize max_pulses for this chunk
    for fname in tqdm(chunk):
        data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
        
        for ev in (go.cxx_api.TofEvent.from_tofpacket(k) for k in data):
            rbids = ev.get_rbids()
            
            for event_id in (ev.get_rbevent(k) for k in rbids):
                if event_id.header.rb_id == 0:
                    continue
                
                key = event_id.header.rb_id
                if len(str(key)) > 2:
                    continue
                
                try:
                    calibration = calib[key]
                except KeyError:
                    continue
                
                voltages = calibration.voltages(event_id)
        
                for channel in event_id.header.get_channels():
                    if channel == 8:
                        continue
                    
                    wv = np.array(voltages[channel]) #calibrating the voltages
                    max_pulse = np.max(wv) 
                    info = {'rbid' : key, 'channel' : channel, 'max_V' : max_pulse}
        
                    max_pulses_chunk.append(info)  # Append to chunk-specific max_pulses
    
    max_pulses.extend(max_pulses_chunk)  # Append chunk-specific results to the main max_pulses list


In [ ]:
print(len(max_pulses))

In [ ]:
for element in tqdm(max_pulses):
    r = element['rbid']
    channel1 = element['channel']
    max_V = element['max_V']

    max_pulse_dict[(r, channel1)].append(max_V)

In [ ]:
print(len(max_pulse_dict.keys()))

In [ ]:
for rb, channel in tqdm(sorted(max_pulse_dict.keys())): #individual plots
    mp = max_pulse_dict[(rb, channel)]
    plt.hist(mp, bins=70)
    plt.title(f' RB {rb}, channel {channel+1} max voltage peak frequency')
    plt.xlabel('Max Voltage Peak [mV]')
    plt.ylabel('count')
    #plt.yscale('log')
    #plt.xlim(-10, 20)
    plt.show()

In [ ]:
#composite plot of most common value from individual plots

from collections import Counter

def most_common_integer(lst):
    # Use Counter to count occurrences of each integer in the list
    counts = Counter(lst)
    # Use the most_common() method to get the most common integer
    most_common = counts.most_common(1)
    # Return the most common integer
    return most_common[0][0]
    
my_dict = {}

for (r, channel1), max_pulse_list in max_pulse_dict.items(): #populate dictionary based on the elements of the baseline_rms_dict. baseline_rms_dict uses the library defaultdict to create a list in baselime_rms_dict for each r,channel1 pair
    common_max_pulse = most_common_integer(max_pulse_list)   # here the mean is calculated of the baseline_rms_list for each dict containing the list of values for each r,channel1 pair.  
    if r in my_dict:
        my_dict[r][channel1] = common_max_pulse

    else:
        my_dict[r]={channel1: common_max_pulse}

# Access all the max_pulse_dict values
max_pulse_values = [max_V for sub_dict in my_dict.values() for max_V in sub_dict.values()]

#filtered_values = np.where(np.array(baseline_rms_values) < 250)[0]

# Create a histogram with values
plt.figure(figsize = (10, 6))
plt.hist(np.array(max_pulse_values), histtype = 'step', bins = 24)

plt.xlabel('Max Voltage Peak [mV]')
plt.ylabel('Count')
plt.title('Max Voltage Peak Distribution, Run 138')
#plt.xticks(np.arange(0.5, 1.5, 0.1))
plt.show()